In [1]:
import os
os.chdir('..')

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import xgboost
from xgboost import XGBClassifier, plot_importance
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, roc_auc_score, f1_score
from sklearn.manifold import TSNE
from sklearn.utils import shuffle
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from scipy.stats import spearmanr

from constants import plots3d_path, persistence_path, landscapes_path
from data_analysis_utils.utils import define_features_var_grouping
from data_analysis_utils.proteins_similarity import pairwise_dist, aggregate_metrics_dim, aggregate_metrics_all, get_samples_low_high_pos_dist, compare_pdb_pairwise_dist
from data_analysis_utils.visualization import apply_mds, compare_pdb_plots

In [5]:
from data_scripts.tda_encoding import get_encoded_proteins_from_coords
from data_scripts.get_protein_data import download_pdb_from_id, extract_3dcoords_from_pdb

In [6]:
def flag_connected_proteins(df_tda_enc):
    # Remove proteins we are not sure if connected by taking a look at persistence in dim 0
    df_tda_enc['connected'] = np.where(((df_tda_enc.end_dim0_0 - df_tda_enc.end_dim0_1) > (df_tda_enc.end_dim0_1 - df_tda_enc.end_dim0_4)), 'Unknown', 'True')
    return df_tda_enc

In [7]:
def get_approximated_land_area(df_tda_enc, dim, land, feat_var_dict_dim_land):
    # Approximate area under land i for dimension 1 and 2
    # Area without scaling on x axis is proportional to area with scaling * domain length - Attention, this is not true for landscapes with multiple peaks -e.g. not connected proteins for which the landscapes is 0 between the peaks!!
    dim_land_shape_vars = [var for var in features_var_dict_dim_land[f'dim{dim}_land{land}'] if ('begin' not in var) and ('end' not in var)]
    df_tda_enc[f'area_dim{dim}_land{land}'] = df_tda_enc[dim_land_shape_vars].apply(lambda x: np.linalg.norm(x),axis=1) * (df_tda_enc[f'end{land-1}_dim{dim}'] - df_tda_enc[f'begin{land-1}_dim{dim}'])
    return df_tda_enc

In [8]:
def interesting_homology_filter(df_tfda_enc, feat_var_dict_dim_land):
    # Infer if protein is connected from 0 dim homology
    df = flag_connected_proteins(df_tfda_enc)
    
    # Get landscape areas: bigger the i-th area bigger the i-th hole
    for dim in range(1,3):
        for land in range(1,4): 
            df = get_approximated_land_area(df_tda_enc=df, dim=dim, land=land, feat_var_dict_dim_land=feat_var_dict_dim_land)

    # Remove proteins that may not be connected
    df = df[df['connected']=='True']
    
    # Remove proteins we are not sure if having interesting dim1 and dim2 homology by removing proteins those with small area under land1
    df = df[(df['area_dim1_land1'] > np.percentile(df['area_dim1_land1'],95)) | (df['area_dim2_land1'] > np.percentile(df['area_dim2_land1'],95))]
    
    return df

In [9]:
def get_similar_protein_pairs(df_tda_enc, pair_dist, different_type_constraint=False, n=5):
    # Get top pairwise distance
    df_pair_dist_top = get_samples_low_high_pos_dist(pair_dist, n=n)
    # Remove duplicates (distance is symmetric)
    df_pair_dist_top_no_dup = df_pair_dist_top.loc[df_pair_dist_top[['pdb_id1','pdb_id2']].apply(set,axis=1).drop_duplicates().index].reset_index(drop=True)
    # Filter to proteins having different types (it's more interesting)
    df_pair_dist_top_no_dup = df_pair_dist_top_no_dup\
        .merge(df_tda_enc[['pdb_id', 'enzyme_type','area_dim1_land1','area_dim1_land2','area_dim1_land3','area_dim2_land1','area_dim2_land2','area_dim2_land3']], left_on='pdb_id1',right_on='pdb_id')\
        .rename(columns={'enzyme_type':'type_1','area_dim1_land1':'area_dim1_land1_pdb1','area_dim1_land2':'area_dim1_land2_pdb1','area_dim1_land3':'area_dim1_land3_pdb1','area_dim2_land1':'area_dim2_land1_pdb1','area_dim2_land2':'area_dim2_land2_pdb1','area_dim2_land3':'area_dim2_land3_pdb1'})\
        .drop(columns='pdb_id')\
        .merge(df_tda_enc[['pdb_id', 'enzyme_type','area_dim1_land1','area_dim1_land2','area_dim1_land3','area_dim2_land1','area_dim2_land2','area_dim2_land3']], left_on='pdb_id2',right_on='pdb_id')\
        .rename(columns={'enzyme_type':'type_2','area_dim1_land1':'area_dim1_land1_pdb2','area_dim1_land2':'area_dim1_land2_pdb2','area_dim1_land3':'area_dim1_land3_pdb2','area_dim2_land1':'area_dim2_land1_pdb2','area_dim2_land2':'area_dim2_land2_pdb2','area_dim2_land3':'area_dim2_land3_pdb2'})\
        .drop(columns='pdb_id')
    if different_type_constraint:
        df_similar_dist = df_pair_dist_top_no_dup[df_pair_dist_top_no_dup['type_1']!=df_pair_dist_top_no_dup['type_2']].sort_values(by='distance').reset_index(drop=True)
    else:
        df_similar_dist = df_similar_dist.sort_values(by='distance').reset_index(drop=True)
    return df_similar_dist


### Load data 

In [13]:
# Load encoded data with enzyme type and pdb_id
df_raw = pd.read_csv('output/tda_encoded_proteins_assembly.csv')
df_raw = shuffle(df_raw, random_state=17).reset_index(drop=True)
df_raw.shape

(14183, 84)

In [14]:
shuffle(df_raw, random_state=17).head()

,end_dim0_4,end_dim0_3,end_dim0_2,end_dim0_1,end_dim0_0,begin0_dim1,end0_dim1,a00_dim1,a10_dim1,b10_dim1,...,b12_dim2,a22_dim2,b22_dim2,a32_dim2,b32_dim2,a42_dim2,b42_dim2,a52_dim2,b52_dim2,pdb_id
6263,0.031053,0.031109,0.031736,0.033875,0.038172,0.041647,0.809293,0.576306,-0.231065,-0.008005,...,-0.001668,0.007618,0.003524,-0.001965,-0.003667,0.000198,0.001975,0.000456,-0.000788,3WWQ
2484,0.037253,0.041352,0.043844,0.049673,0.055790,0.005469,0.734540,0.032304,0.014172,0.022359,...,0.011251,-0.010462,-0.004101,-0.000637,-0.000441,0.001366,0.001842,0.000345,-0.000658,6FJF
9128,0.012660,0.013713,0.013939,0.014772,0.016293,0.009876,0.431716,0.052027,0.008269,0.041952,...,0.002368,0.000154,-0.005473,-0.002060,0.001841,-0.000313,0.000148,-0.000083,0.000581,1KTL
8878,0.016735,0.016945,0.018145,0.020255,0.021961,0.032096,1.079295,0.252217,0.070922,0.011673,...,0.017554,-0.007181,-0.009053,-0.001113,-0.002425,-0.000719,0.001023,0.001347,0.001480,7TN9
10632,0.016606,0.016822,0.017014,0.017511,0.018205,0.141734,4.260686,3.138791,-1.243535,-0.008632,...,-0.118243,0.049629,0.016348,-0.038156,-0.019759,0.031078,0.022973,-0.001198,-0.001228,7WZ3


In [15]:
df_raw.head()

,end_dim0_4,end_dim0_3,end_dim0_2,end_dim0_1,end_dim0_0,begin0_dim1,end0_dim1,a00_dim1,a10_dim1,b10_dim1,...,b12_dim2,a22_dim2,b22_dim2,a32_dim2,b32_dim2,a42_dim2,b42_dim2,a52_dim2,b52_dim2,pdb_id
0,0.016653,0.016829,0.017497,0.018127,0.020563,0.008818,2.287163,0.397747,0.008854,0.212187,...,-0.001136,-0.015003,0.015701,0.013206,-0.006437,-0.023021,-0.014598,-0.011707,0.013695,5N60
1,0.013948,0.018293,0.018507,0.019814,0.020948,0.015655,0.322974,0.116917,-0.042125,0.051454,...,-0.003463,-0.003617,-0.012425,-0.003859,0.003016,0.001733,-0.000613,-0.001436,-0.000055,1OED
2,0.028250,0.028999,0.029191,0.029231,0.033350,0.005733,0.924789,0.010331,0.006648,0.005211,...,0.007018,-0.000143,-0.006174,0.001364,0.001377,-0.002091,-0.000105,0.001839,0.001000,4AC7
3,0.017916,0.018095,0.018544,0.018621,0.991182,0.514619,3.548179,1.570846,-0.679233,-0.455567,...,-0.000155,-0.020321,0.010022,0.014369,-0.002203,-0.001113,-0.011260,-0.001214,0.006310,6QV0
4,0.043156,0.055253,0.056881,0.101645,0.138809,0.068090,2.583287,0.718226,-0.116782,0.383141,...,0.001824,0.002775,0.002837,0.001003,0.003037,-0.000466,0.002592,-0.001430,0.002117,4UJ3


In [12]:
# Load table with uniprot id, pdb id and Enzyme Classification number
df_pue = pd.read_csv('data/pdb_uniprot_ec.csv')
df_pue.shape

(85527, 3)

In [ ]:
# Add uniprot id and EC number info to encoded data 
df_puet = df_raw.merge(df_pue)
df_puet.shape

In [ ]:
# Restrict to enzymes having EC number (it seems a lot do not have it recorded in pdb file) and uniprot id (few don't have)
df_with_ec = df_puet.dropna().reset_index(drop=True)
print(df_with_ec.shape)

In [ ]:
# In very few cases EC number doesn't correspond with enzyme type. Let's filter out those samples
df = df_with_ec[df_with_ec['enzyme_type'].apply(lambda x: str(enzyme_ec_label_dict[x])) == df_with_ec['ec_num'].apply(lambda x: x[0])]
print(df.shape)

In [ ]:
# EC number distribution
df['ec_num'].value_counts().describe()

In [ ]:
# Enzyme types distribution
df['enzyme_type'].value_counts()

In [ ]:
uniprot_val_data = get_external_validation_pdb_ids()

In [ ]:
# Take out external validation data
df_external_val = df[df['uniprot_id'].isin(uniprot_val_data)].reset_index(drop=True)
df = df[df['uniprot_id'].isin(uniprot_val_data)==False].reset_index(drop=True)

### Data analysis 

In [ ]:
df.head(3)

In [ ]:
df.describe()

In [ ]:
grouped_feat_var = define_features_var_grouping(df)
feat_cols = grouped_feat_var['feat_cols']
feat_cols_dim0 = grouped_feat_var['feat_cols_dim0']
feat_cols_begin_end = grouped_feat_var['feat_cols_begin_end']
features_var_dict_dim_land_shape_extrema = grouped_feat_var['features_var_dict_dim_land_shape_extrema']
features_var_dict_dim_land = grouped_feat_var['features_var_dict_dim_land']
features_var_dict_dim = grouped_feat_var['features_var_dict_dim']

In [ ]:
# Check correlation among group of features
for key in features_var_dict_dim_land.keys():
    df_corr = df[np.sort(features_var_dict_dim_land[key])].corr('spearman')
    sns.heatmap(df_corr, annot = True, fmt = '.2f')
    print(key)
    plt.show()

In [ ]:
# Considerations:
# - dim 0 features are very correlated, what about taking only 0, 2 and 4?
# - begin_dim1 and end_dim1 are correlated, while Fourier coefficients are not (makes sense)
# - a10 is bigger than b10 for first landscape since for the approximated function we have f(0)=0 like sin, 
# - what if we take fewer Fourier coefficients? 3-series instead of 5?

In [ ]:
# Check correlation with outcome
y = df['enzyme_type'].apply(lambda x: enzyme_ec_label_dict[x]).values
corrs = []
for var in feat_cols:
    corrs += [spearmanr(df[var],y)[0]]
df_corr_outcome = pd.DataFrame(zip(feat_cols,corrs),columns=['feat','corr']).sort_values(by='corr',ascending=False)

In [ ]:
df_corr_outcome.head(15)

#### Train-test split and outliers removal 

In [ ]:
# split data into train and test sets
seed = 7
test_size = 0.2
df_train, df_test = train_test_split(df, test_size=test_size, stratify=df['enzyme_type'], random_state=seed)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
x_train = df_train[feat_cols]
x_test = df_test[feat_cols]
y_train = df_train['enzyme_type'].apply(lambda x: enzyme_ec_label_dict[x]).values
y_test = df_test['enzyme_type'].apply(lambda x: enzyme_ec_label_dict[x]).values

In [ ]:
# identify outliers in the training dataset based on begin and end of 0,1,and 2 dim features (this should hint errors in recorded coordinates and the resulting distributions looks nicer than if we remove only dim0 features)
# contamination gives the proportion of outliers we wish to remove, default is "auto"
iso_forest = IsolationForest(contamination=0.1) # contamination was set so that the feature distribution histograms are close to normal
yhat = iso_forest.fit_predict(x_train[feat_cols_begin_end])

In [ ]:
# select all rows that are not outliers
mask = yhat != -1
print("num original samples:", len(x_train))
print("num samples after removing outliers:", sum(mask))

In [ ]:
# Remove outliers
x_train_no_out, y_train_no_out = x_train[mask], y_train[mask]

In [ ]:
# Check features distribution
for key in features_var_dict_dim_land.keys():
    x_train[features_var_dict_dim_land[key]].hist(figsize=(10,10))
    print(key)
    plt.show()

In [ ]:
# Check features distribution after removing outliers
for key in features_var_dict_dim_land.keys():
    x_train_no_out[features_var_dict_dim_land[key]].hist(figsize=(10,10))
    print(key)
    plt.show()
# Now features distribution makes more sense

In [ ]:
# Prevalence with and without outlier is similar
df_enz_type_prev = df['enzyme_type'].value_counts()/len(df['enzyme_type']) * 100
df_enz_type_prev_no_out = pd.DataFrame(y_train_no_out).value_counts()/len(y_train_no_out) * 100
print(df_enz_type_prev)
print(df_enz_type_prev_no_out)

In [ ]:
# let's look how extreme outliers look like
iso_forest_extreme_outlier= IsolationForest(contamination=0.0001)
yhat_extreme_outlier = iso_forest_extreme_outlier.fit_predict(x_train[feat_cols_dim0])

In [ ]:
df_extreme_out = df_train[yhat_extreme_outlier==-1]
# todo: check 3d plot of outliers...

In [ ]:
# Many proteins have trivial global shapes so we can't conclude much from persistence homology.
# Let's look for proteins having one single connected component and non-trivial homology in dim 1 and 2.
# Anyway keep in mind all knots in 3d are homotopy equivalent, so we should not be able to tell the difference between different knotted chains.

In [ ]:
df_no_out = df.loc[x_train_no_out.index]

In [ ]:
df_fun_homology = interesting_homology_filter(df_tfda_enc=df_no_out, feat_var_dict_dim_land=features_var_dict_dim_land)

In [ ]:
df_fun_homology

#### Pairwise distance analysis

In [ ]:
# Restrict to smaller dataset to allow faster computation of pairwise distances
df_small = df.loc[df_fun_homology.index]

In [ ]:
# Get pairwise distance at different aggregation levels
pair_dist_dict = pairwise_dist(df=df_small, dict_features_var_dims_lands=features_var_dict_dim_land_shape_extrema)
pair_dist_dict_dim = aggregate_metrics_dim(pair_dist_dict)
pair_dist_final = aggregate_metrics_all(pair_dist_dict_dim)

In [ ]:
#df_dim0_close_pairs = get_samples_low_high_pos_dist(pair_dist_dict_dim['dim0'], n=5, low=True).drop_duplicates().rename(columns={'distance':'distance0'})

In [ ]:
#df_dim1_close_pairs = get_samples_low_high_pos_dist(pair_dist_dict_dim['dim1'], n=5, low=True).drop_duplicates().rename(columns={'distance':'distance1'})

In [ ]:
#df_dim2_close_pairs = get_samples_low_high_pos_dist(pair_dist_dict_dim['dim2'], n=5, low=True).drop_duplicates().rename(columns={'distance':'distance2'})

In [ ]:
#df_dim1_close_dim2_close = df_dim1_close_pairs.merge(df_dim2_close_pairs).sort_values(by='distance1',ascending=False).reset_index(drop=True)

In [ ]:
#df_dim1_close_dim2_close

In [ ]:
df_fun_homology

In [ ]:
dg = get_similar_protein_pairs(df_fun_homology, pair_dist_final, n=5)

In [ ]:
dg.sort_values(by=['area_dim1_land3_pdb1','pdb_id1','distance'],ascending=[False,False,True])#.head(20)
# nice example:7CW2, 7CW3, 7CVZ, 6Z6O, 3J0F, 3J0C are similar viruses - Chikungunya,  Sindbis virion and Venezuelan Equine Encephalitis Virus


In [ ]:
df_small

In [ ]:
get_similar_protein_pairs(pair_dist_dict_dim['dim1'], n=5)

In [ ]:
i = 2728

In [ ]:
for i in range(2727,2730):
    sim_pair = list(df_similar_dist_diff_type.reset_index(drop=True)[['pdb_id1','pdb_id2']].loc[i].values)
    compare_pdb_pairwise_dist(pdb_id1=sim_pair[0], pdb_id2=sim_pair[1], dict_dist_no_aggr=pair_dist_dict, dict_dist_aggr_dim=pair_dist_dict_dim, dist_aggr_global=pair_dist_final)

In [ ]:
sim_pair = list(df_similar_dist_diff_type.reset_index(drop=True)[['pdb_id1','pdb_id2']].loc[i].values)

In [ ]:
j=6

In [ ]:
sim_pair = list(df_dim0_close_dim1_far[['pdb_id1','pdb_id2']].loc[j].values)

In [ ]:
sim_pair = ['4YMK','4EKZ']

In [ ]:
# 4TOB (big dim2 hole) and 2P0I (small dim2 hole)

In [ ]:
compare_pdb_pairwise_dist(pdb_id1=sim_pair[0], pdb_id2=sim_pair[1], dict_dist_no_aggr=pair_dist_dict, dict_dist_aggr_dim=pair_dist_dict_dim, dist_aggr_global=pair_dist_final)

In [ ]:
# 3d plots from pdb website are nicer and also seem a bit different..is it or is it just a matter of perspective?

In [ ]:
# nice examples: 4A9G (sphere), 7CM5 (torus), 

#### Data visualization 

In [ ]:
compare_pdb_plots(pdb_list=sim_pair)

In [ ]:
# look for examples with high dim1/2 difference and low dim0 diff
# what's persistence homology of a 3d knot?

In [ ]:
stop

In [ ]:
# Use Multi Dimensional Scaling on pairwise distances to visualize data in 3d
# This is done at different aggregation levels
# No aggregation
for k, v in zip(pair_dist_dict.keys(), pair_dist_dict.values()):
    apply_mds(title=k,df_dissim=v,labels=df_small['enzyme_type'],enzyme_color_dict=enzyme_ec_label_color_dict)
# Aggregate at dim level
for k, v in zip(pair_dist_dict_dim.keys(), pair_dist_dict_dim.values()):
    apply_mds(title=k,df_dissim=v,labels=df_small['enzyme_type'],enzyme_color_dict=enzyme_ec_label_color_dict)
# Aggregate all dims and all lands
apply_mds(title='all dim',df_dissim= pair_dist_final,labels=df_small['enzyme_type'],enzyme_color_dict=enzyme_ec_label_color_dict)

In [ ]:
# todo: reduce size of created pngs

In [ ]:
# it seems hydrolases can have multiple 3dim holes while other enzymes usually do not

In [ ]:
stop

#### Classification model

In [ ]:
# feature selection? are all the 3 landscapes and 5 fourier coefficients useful?

In [ ]:
# scale_pos_weight = y_text.value_counts()[0]/y_text.value_counts()[1]

In [ ]:
#from scipy.stats import loguniform
#loguniform(1e-6, 1e-3).rvs()

In [ ]:
# declare cv parameters
cv_params = {
    'learning_rate': [0.1, 0.01, 0.001],
    'gamma': np.linspace(0,0.5,5),
    'max_depth': [10, 15, 25],
    'min_child_weight': [1,2,5],
    'subsample':np.linspace(0.5,1,5),
    'n_estimators':[100, 200, 500, 1000],
        }    

# add scale_pos weight
# eta is same as learning_rate
# "objective":"binary:logistic"

In [ ]:
xgb_classifier = xgboost.XGBClassifier()
xgb_classifier_cv = RandomizedSearchCV(xgb_classifier,param_distributions=cv_params,n_iter=100,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [ ]:
# Run CV (takes some time) and store best parameters as json
# xgb_classifier_cv.fit(x_train, y_train)
# best_classifier = xgb_classifier_cv.best_estimator_
# best_classifier_params = xgb_classifier_cv.best_estimator_.get_params()
#with open("output/xgboost_cv_best_params.json", "w") as f:
#    json.dump(best_params , f)

In [ ]:
stop

In [ ]:
#best_xgboost_params_cv_path = "output/xgboost_cv_best_params.json"
#best_xgboost_params_cv = json.load(open(best_xgboost_params_cv_path))

In [ ]:
xgb_best = xgboost.XGBClassifier(**best_xgboost_params_cv)

In [ ]:
xgb_best.fit(x_train_no_out,y_train_no_out -1)

In [ ]:
# make predictions for test data
y_pred = xgb_best.predict(x_test)

In [ ]:
y_pred_proba = xgb_best.predict_proba(x_test)

In [ ]:
enzyme_ec_label_dict

In [ ]:
df_test['enzyme_type'].value_counts()

In [ ]:
# evaluate predictions
confusion_matrix(y_true=y_test-1,y_pred=y_pred)

In [ ]:
# add weights?

In [ ]:
roc_auc_score(y_true=y_test-1,y_score=y_pred_proba, average='weighted', multi_class='ovr')

In [ ]:
f1_score(y_true=y_test-1,y_pred=y_pred, average='weighted')

In [ ]:
matthews_corrcoef(y_true=y_test-1,y_pred=y_pred)

In [ ]:
# - five 0-dim bars
# - begin/end domain of land 1, followed by 1 + 5*2 fourier coeff a0,a1,b1,a2,b2,... repeated num_1dim_landscapes time
# - begin/end domain of land 2, followed by 1 + 5*2 fourier coeff a0,a1,b1,a2,b2,... repeated num_2dim_landscapes time

In [ ]:
# plot_importance(xgb_best,max_num_features=10)

In [ ]:
df_feat_imp = pd.DataFrame(xgb_best.feature_importances_, columns=['feat_imp'], index=x_train.columns)

In [ ]:
num_top_feat = 10
num_bottom_feat = 10

In [ ]:
df_top_feat = df_feat_imp.sort_values(by='feat_imp',ascending=False).head(num_top_feat)
df_bottom_feat = df_feat_imp.sort_values(by='feat_imp',ascending=True).head(num_bottom_feat)

In [ ]:
plt.barh(range(num_top_feat), df_top_feat['feat_imp'].values[::-1])
y_pos = np.arange(num_top_feat)
plt.yticks(y_pos, df_top_feat.index[::-1])
plt.show()

In [ ]:
plt.barh(range(num_top_feat), df_bottom_feat['feat_imp'].values[::-1])
y_pos = np.arange(num_top_feat)
plt.yticks(y_pos, df_bottom_feat.index[::-1])
plt.show()

In [ ]:
# fit model no training data. linear models don't work well
# model = LinearSVC(class_weight='balanced') - bad
# model = LogisticRegression(class_weight='balanced') - bad
# https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning
# add cv together with max_depth increase
# model.fit(x_train, y_train)

In [ ]:
#data_dmatrix = xgboost.DMatrix(data=x_train,label=y_train)
#xgb_evaluation_cv = cv(dtrain=data_dmatrix, params=best_classifier_params, nfold=5, metrics="auc", as_pandas=True, seed=123,num_boost_round=15)
#xgb_evaluation_cv.head()

In [ ]:
stop

In [ ]:
# Look for pairs of similar proteins. Are there any couples having similar shape but not similar sequence? some algorithms for protein clasification are based on sequence matching, is it enough? this could be better checked using bottleneck distance, but it would be more time consuming and be a different topic

In [ ]:
from scipy.spatial.distance import pdist, squareform

In [ ]:
distances = pdist(x, metric='euclidean')
dist_mat = squareform(distances)

In [ ]:
df0[df0['pdb_id'] == '102L']

In [ ]:
df0[df0['pdb_id'] == '4EW4']

In [ ]:
dist_mat_non_zero = dist_mat + np.identity(dist_mat.shape[0])

In [ ]:
min_pairwise_dist_pdb_id1 = np.argmin(np.min(dist_mat_non_zero,axis=1))

In [ ]:
min_pairwise_dist_pdb_id2 = np.argmin(dist_mat_non_zero[min_pairwise_dist_pdb_id1])

In [ ]:
print(df0.loc[min_pairwise_dist_pdb_id1]['pdb_id'])
print(df0.loc[min_pairwise_dist_pdb_id2]['pdb_id'])

In [ ]:
pd.DataFrame(dist_mat_non_zero).describe()

In [ ]:
min_dist_pdb_id1 = np.argmin(dist_mat_non_zero[0,:])

In [ ]:
print(df0.loc[0]['pdb_id'])
print(df0.loc[min_dist_pdb_id1]['pdb_id'])

In [ ]:
dist_mat_non_zero[min_pairwise_dist_pdb_id1,min_pairwise_dist_pdb_id2]

In [ ]:
# 0 distance....are there duplicates??

In [ ]:
# no surprise they are similar, one is a mutation

In [ ]:
""" 
# TSNE
#tsne_result = TSNE(n_components=2, learning_rate='auto',init='random', perplexity=3).fit_transform(x_train_no_out)
# Plot the result of our TSNE with the label color coded
# A lot of the stuff here is about making the plot look pretty and not TSNE
tsne_result_df = pd.DataFrame({'tsne_1': tsne_result[:,0], 'tsne_2': tsne_result[:,1], 'label': y_train_no_out})
fig, ax = plt.subplots(1)
sns.scatterplot(x='tsne_1', y='tsne_2', hue='label', data=tsne_result_df, ax=ax,s=120)
lim = (tsne_result.min()-5, tsne_result.max()+5)
ax.set_xlim(lim)
ax.set_ylim(lim)
ax.set_aspect('equal')
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)"""

In [ ]:
"""# PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x_train_no_out)
# Apply PCA for data visualization
# do not scale!
pca = PCA(n_components = 2)
data_pca = pca.fit_transform(x_train_no_out)
data_pca = pd.DataFrame(data_pca,columns=['PC1','PC2'])
data_pca['enzyme_type'] = y_train_no_out
sns.scatterplot(data=data_pca, x="PC1", y="PC2", hue="enzyme_type",alpha=0.7)"""